In [ ]:
from flask import Flask, render_template, request, redirect, url_for
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
import ssl
import logging
import os

# Setup Flask App
app = Flask(__name__, static_folder="static", template_folder="templates")
app.secret_key = "supersecretkey"  # Required for session management

# Enable Logging
logging.basicConfig(filename="server.log", level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Flask-Login Setup
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = "login"  # Redirects unauthenticated users

# Mock User Database
users = {"admin": {"password": "1234"}}  # Simple user storage

# User Class
class User(UserMixin):
    def __init__(self, username):
        self.id = username

@login_manager.user_loader
def load_user(user_id):
    return User(user_id) if user_id in users else None

# Route: Homepage
@app.route("/")
def home():
    return render_template("index.html")

# Route: Login Page
@app.route("/login", methods=["GET", "POST"])
def login():
    if request.method == "POST":
        username = request.form["username"]
        password = request.form["password"]
        if username in users and users[username]["password"] == password:
            user = User(username)
            login_user(user)
            logging.info(f"User {username} logged in.")
            return redirect(url_for("dashboard"))
        else:
            logging.warning("Failed login attempt")
    return render_template("login.html")

# Route: Protected Dashboard
@app.route("/dashboard")
@login_required
def dashboard():
    return f"Welcome {current_user.id}, this is your secure dashboard!"

# Route: Logout
@app.route("/logout")
@login_required
def logout():
    logging.info(f"User {current_user.id} logged out.")
    logout_user()
    return redirect(url_for("login"))

# Run HTTPS Server
if __name__ == "__main__":
    context = ("cert.pem", "key.pem")  # SSL certificate & key (see below)
    app.run(host="localhost", port=8080, ssl_context=context)

    

In [6]:
# simplest version backup
from flask import Flask, render_template
import time
from datetime import datetime

host_name = 'localhost'
port_number = 8080
app = Flask(__name__, static_folder="static", template_folder="templates")

@app.route("/")
def home():
    return render_template("index.html")


if __name__ == "__main__":
    # record start time
    start_timestamp = time.time()
    start_time = datetime.fromtimestamp(start_timestamp).strftime('%Y-%m-%d %H:%M:%S')
    print(f"sever starts at {start_time}")

    try:
        context = ("cert.pem", "key.pem")  # Use the generated SSL certificate
        app.run(host=host_name, port=port_number, ssl_context=context)
    except KeyboardInterrupt:
        pass

    # record end time
    end_timestamp = time.time()
    end_time = datetime.fromtimestamp(end_timestamp).strftime('%Y-%m-%d %H:%M:%S')
    print(f"sever ends at {end_time}")

sever starts at 2025-03-04 22:21:47
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://localhost:8080
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

e:\Miniforge\envs\Csecurity\Lib\site-packages\IPython\core\interactiveshell.py:3554: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [7]:
# frontend backup
from flask import Flask, render_template, request, redirect, url_for, session, flash, jsonify
import time
from datetime import datetime
import os


host_name = 'localhost' # Must be 'localhost' for https
port_number = 8080 # 8080 for local host or 5000 (flask defualt port number)
app = Flask(__name__, static_folder="static", template_folder="templates")

app.secret_key = 'my_secret_key' # unsecure key only for test

# read users' names and passwords
users = {"admin": {"password": "1234"}}  


SESSION_TIMEOUT = 20  # after this time of no activities, log the user out 

@app.before_request
def enforce_session_timeout():
    """Check if the session has expired"""
    if 'username' in session:
        last_activity = session.get("last_activity", time.time())
        if time.time() - last_activity > SESSION_TIMEOUT:
            session.pop("username", None)  # Remove user session
            session.pop("last_activity", None)
            flash("Session expired. Please log in again.", "warning")
            return redirect(url_for("login"))

        session["last_activity"] = time.time()  # Update last activity timestamp


@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        # record valid username
        if username in users and users[username]['password'] == password:
            session['username'] = username
            session["last_activity"] = time.time()
            flash("Login successful!", "success")
            return redirect(url_for('home'))
        else:
            flash("Invalid credentials, please try again.", "error")
    return render_template('login.html') 

@app.route("/update_activity", methods=["POST"])
def update_activity():
    """Update session activity when user interacts."""
    if "username" in session:
        session["last_activity"] = time.time()
        return jsonify({"status": "success"})
    return jsonify({"status": "error"}), 403

@app.route("/logout")
def logout():
    session.pop("username", None)
    session.pop("last_activity", None)
    flash("You have been logged out.", "info")
    return redirect(url_for("login"))

@app.route("/")
def home():
    # Check if there is a valid user. If so, go to the home page.
    if 'username' not in session:
        return redirect(url_for('login'))
    return render_template("index.html")

if __name__ == "__main__":
    # record start time
    start_timestamp = time.time()
    start_time = datetime.fromtimestamp(start_timestamp).strftime('%Y-%m-%d %H:%M:%S')
    print(f"server starts at {start_time}")

    try:
        context = ("cert.pem", "key.pem")  # Use the generated SSL certificate
        app.run(host=host_name, port=port_number, ssl_context=context)
    except KeyboardInterrupt:
        pass

    # record end time
    end_timestamp = time.time()
    end_time = datetime.fromtimestamp(end_timestamp).strftime('%Y-%m-%d %H:%M:%S')
    print(f"server ends at {end_time}")

b':\xb0\x99\x83#\xc4\xc5{\xf4\x13b\xc3\x1e`\xeeY\x81\x02\xe1.k\x01\xc2\x14'


In [2]:
import sqlite3
bank_database = "bank.db"
def get_db_connect():
    """Establish a connection to the SQLite database."""
    conn = sqlite3.connect(database=bank_database)
    conn.row_factory = sqlite3.Row
    return conn

conn = get_db_connect()
users = conn.execute("SELECT * FROM users").fetchall()
    
for i in users:
    print(i['balance'])

40.0
500.0
1010.0
120.0


In [3]:
import sqlite3
bank_database = "bank.db"
def get_db_connect():
    """Establish a connection to the SQLite database."""
    conn = sqlite3.connect(database=bank_database)
    conn.row_factory = sqlite3.Row
    return conn

conn = get_db_connect()
users = conn.execute("SELECT * FROM users").fetchall()
    
for i in users:
    print(i['id'])
    print(type(i['id']))

1001
<class 'int'>
1002
<class 'int'>
1003
<class 'int'>
1004
<class 'int'>
